In [1]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset='all', random_state=156)

In [2]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [3]:
import pandas as pd

print('target 클래스의 값과 분포도 \n',pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n', news_data.target_names)
len(news_data.target_names), pd.Series(news_data.target).shape

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


(20, (18846,))

In [4]:
from sklearn.datasets import fetch_20newsgroups

train_news = fetch_20newsgroups(subset='train',remove=('headers','footers','quotes'),random_state=156)
X_train = train_news.data
y_train = train_news.target

test_news= fetch_20newsgroups(subset='test',remove=('headers', 'footers','quotes'),random_state=156)
X_test = test_news.data
y_test = test_news.target

print(len(train_news.data),len(test_news.data))

11314 7532


In [5]:
from sklearn.feature_extraction.text import CountVectorizer

cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

X_test_cnt_vect = cnt_vect.transform(X_test)

print(X_train_cnt_vect.shape, X_test_cnt_vect.shape)

(11314, 101631) (7532, 101631)


In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect,y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print(accuracy_score(y_test,pred))

0.6074083908656399


C:\Users\Lee\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print(accuracy_score(y_test, pred))

0.6736590546999469


In [9]:
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred= lr_clf.predict(X_test_tfidf_vect)
print(accuracy_score(y_test,pred))

0.6922464152947424


In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'C':[0.01,0.1,1,5,10]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid = params,cv=3)
grid_cv_lr.fit(X_train_tfidf_vect,y_train)
print(grid_cv_lr.best_params_)

pred = grid_cv_lr.predict(X_test_tfidf_vect)
print(accuracy_score(y_test, pred))

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('tfidf_vect',TfidVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)),('lr_clf',LogisticRegression(C=10))])

pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
print(accuracy_score(y_test,pred))

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('tfid_vect',TfidVectorizer(stop_words='english')),('lr_clf',LogisticRegression())])

params = { 'tfidf_vect__ngram_range':[(1,1),(1,2),(1,3)]
           'tfidf_vect__max_df' :[100,300,700],
         'lr_clf__C':[1,5, 10]}

grid_cv_pipe = GridSearchCV(pipeline, param_grid = params, cv=3)
grid_cv_pipe.fit(X_train, y_train)
print(grid_cv_pipe.best_params_)
pred = grid_cv_pipe.predict(X_test)
print(accuracy_score(y_test,pred))